In [ ]:
# 伪代码和概念结构

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_neo4j import Neo4jGraph

# 1. 初始化模型和图数据库连接
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,  # 设置温度以控制输出的随机性
    base_url="",
    api_key="",
)

In [6]:
from llmsplitter import LLMStructuralTextSplitter,ConcurrentHierarchicalSplitter

coarse_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,          # 可以适当增大块大小
    chunk_overlap=200,        # <-- 设置一个足够大的重叠区
    keep_separator=True,
    is_separator_regex=False
)

fine_splitter = LLMStructuralTextSplitter(llm=llm)

concurrent_splitter = ConcurrentHierarchicalSplitter(
    coarse_splitter=coarse_splitter,
    fine_splitter=fine_splitter,
    max_concurrency=10,
)

In [7]:
# 3. 加载和处理文档
# 加载所有方志和诗词的txt文件
from langchain.document_loaders import TextLoader
loader = TextLoader(file_path="./data_simplified.txt", encoding="utf-8")
documents = loader.load()

In [8]:
docs=concurrent_splitter.split_documents(documents)

--- 步骤 1: 正在进行重叠预分块... ---
--- 预分块完成，共得到 71 个重叠的巨型块。 ---

--- 步骤 2: 正在并发处理所有块 (并发上限: 10)... ---
--- 并发处理完成。 ---

--- 步骤 3: 正在进行结果的裁剪、去重与错误处理... ---
  [警告] 第 29 个块处理失败，错误类型: ContentFilterFinishReasonError。将保留原始粗分块内容。
  [警告] 第 51 个块处理失败，错误类型: ContentFilterFinishReasonError。将保留原始粗分块内容。
--- 裁剪、去重与错误处理完成。 ---


In [9]:
import json
from pathlib import Path
from typing import List
from langchain_core.documents import Document

output_dir = Path("split_outputs_jsonl")
output_dir.mkdir(exist_ok=True)
output_filename = "processed_chunks2.jsonl"
output_path = output_dir / output_filename

# 2. 以写入模式打开文件
try:
    with open(output_path, "w", encoding="utf-8") as f:
        # 遍历每一个 Document 对象
        for doc in docs:
            # a. 将 Document 对象转换为一个字典
            data_record = {
                "page_content": doc.page_content,
                "metadata": doc.metadata
            }
            
            # b. 使用 json.dumps 将字典序列化为 JSON 字符串
            #    ensure_ascii=False 对于正确保存中文字符至关重要
            json_string = json.dumps(data_record, ensure_ascii=False)
            
            # c. 将 JSON 字符串写入文件，并在末尾加上换行符
            f.write(json_string + "\n")

    print(f"--- 所有文档块已成功保存到 -> {output_path} ---")

except IOError as e:
    print(f"--- 写入文件时发生错误: {output_path}. 错误详情: {e} ---")

--- 所有文档块已成功保存到 -> split_outputs_jsonl/processed_chunks2.jsonl ---


In [10]:
# --- 从 JSONL 文件加载回 Document 列表的代码 ---

# 1. 定义要读取的文件路径
input_path = Path("split_outputs_jsonl/processed_chunks2.jsonl")

# 2. 准备一个空列表来存放加载后的 Document 对象
loaded_documents: List[Document] = []

print(f"\n--- 开始从文件加载 Document 块: {input_path} ---")

# 3. 检查文件是否存在
if not input_path.exists():
    print(f"--- 错误：文件不存在 -> {input_path} ---")
else:
    try:
        # 以读取模式打开文件
        with open(input_path, "r", encoding="utf-8") as f:
            # 逐行读取文件
            for line in f:
                # a. 使用 json.loads 将每一行的 JSON 字符串解析为字典
                data_record = json.loads(line)
                
                # b. 使用字典中的数据重新创建 Document 对象
                doc = Document(
                    page_content=data_record["page_content"],
                    metadata=data_record["metadata"]
                )
                
                # c. 将创建好的 Document 对象添加到列表中
                loaded_documents.append(doc)

        print(f"--- 加载完成！成功从文件恢复了 {len(loaded_documents)} 个 Document 对象。 ---")

        # 验证一下加载回来的第一个 Document
        if loaded_documents:
            print("\n验证加载的第一个文档块:")
            first_doc = loaded_documents[0]
            print(f"内容:\n---\n{first_doc.page_content[:100]}...\n---")
            print(f"元数据: {first_doc.metadata}")
            
    except (IOError, json.JSONDecodeError) as e:
        print(f"--- 读取或解析文件时发生错误: {input_path}. 错误详情: {e} ---")


--- 开始从文件加载 Document 块: split_outputs_jsonl/processed_chunks2.jsonl ---
--- 加载完成！成功从文件恢复了 432 个 Document 对象。 ---

验证加载的第一个文档块:
内容:
---
永乐大典卷之二千二百六十八【六模】
湖
巢湖
《合肥志》
在合肥县东南六十里。亦名焦湖。汉明帝十一年。漅湖出黄金。庐江太守以献。漅。音勦或曰勦湖。俗讹为焦湖。以其在巢县。亦曰巢湖。周围四百里。港（汊）...
---
元数据: {'source': './data_simplified.txt'}
